The 2023 Ironman was the 2nd triatholon I've done in my life, the first being a sprint event in rural maryland that I had not trained for (highlighted with a 400m pool swim with >50 contestants). With that in mind I had to create a training schedule. 

In [28]:
import pandas as pd
import matplotlib.pyplot as plt

# Base path where your data is located
base_path = '/Users/ngirmay/Documents/GitHub/ironman_retrospective/apple_health/health_data_exported/'

# Specific file for body fat percentage data
body_fat_file = 'HKQuantityTypeIdentifierBodyFatPercentage_2024-04-99_18-30-56_SimpleHealthExportCSV.csv'

# Full path to the body fat percentage file
body_fat_file_path = base_path + body_fat_file

df_body_fat = pd.read_csv(body_fat_file_path, sep=',', skiprows=1)  # Skipping the first row (sep=)

# Print the columns to check if it’s loaded correctly
print(df_body_fat.columns)



Index(['type', 'sourceName', 'sourceVersion', 'productType', 'device',
       'startDate', 'endDate', 'unit', 'value'],
      dtype='object')


In [29]:
import pandas as pd
import plotly.express as px

# Base path where your data is located
base_path = '/Users/ngirmay/Documents/GitHub/ironman_retrospective/apple_health/health_data_exported/'

# Specific file for body fat percentage data
body_fat_file = 'HKQuantityTypeIdentifierBodyFatPercentage_2024-04-99_18-30-56_SimpleHealthExportCSV.csv'

# Full path to the body fat percentage file
body_fat_file_path = base_path + body_fat_file

# Load the dataset
df_body_fat = pd.read_csv(body_fat_file_path, sep=',', skiprows=1)

# Convert 'startDate' to datetime format
df_body_fat['startDate'] = pd.to_datetime(df_body_fat['startDate'])

# Multiply 'value' by 100 to convert it into a percentage for better readability
df_body_fat['percentage_value'] = df_body_fat['value'] * 100

# Apply the Coolors color scheme
colors = {
    'line': '#E63946',  # Red for the line
    'background': '#F1FAEE',  # Light cream for the background
    'grid': '#A8DADC',  # Light teal for grid lines
    'title': '#1D3557',  # Dark blue for title
    'axes': '#457B9D'  # Steel blue for axes labels
}

# Create the interactive line plot with Plotly
fig = px.line(df_body_fat, x='startDate', y='percentage_value', 
              title='Body Fat Percentage Over Time', 
              labels={'percentage_value': 'Body Fat Percentage (%)', 'startDate': 'Date'})

# Update layout to match the color scheme and center the title
fig.update_layout(
    plot_bgcolor=colors['background'],  # Set background color
    paper_bgcolor=colors['background'],  # Paper (overall background)
    title_font=dict(size=24, color=colors['title'], family="Arial"),  # Title font styling (Arial is already bold by default)
    title_x=0.5,  # Center the title
    xaxis=dict(tickformat='%Y-%m-%d', gridcolor=colors['grid'], tickangle=-45),  # Combine x-axis settings
    yaxis=dict(gridcolor=colors['grid']),  # Set gridline color for y-axis
    font=dict(color=colors['axes']),  # Set font color for axes labels
    xaxis_title='Date',
    yaxis_title='Body Fat Percentage (%)',
    showlegend=False  # Hide the legend
)

# Update line color
fig.update_traces(line_color=colors['line'], line_width=3)

# Show interactive plot
fig.show()


In [30]:
import pandas as pd
import plotly.express as px

# Base path where your data is located
base_path = '/Users/ngirmay/Documents/GitHub/ironman_retrospective/apple_health/health_data_exported/'

# Specific file for body mass data
body_mass_file = 'HKQuantityTypeIdentifierBodyMass_2024-04-99_18-30-56_SimpleHealthExportCSV.csv'

# Full path to the body mass file
body_mass_file_path = base_path + body_mass_file

# Load the dataset for body mass
df_body_mass = pd.read_csv(body_mass_file_path, sep=',', skiprows=1)

# Convert 'startDate' to datetime format
df_body_mass['startDate'] = pd.to_datetime(df_body_mass['startDate'])

# Apply the Coolors color scheme
colors = {
    'line': '#E63946',  # Red for the line
    'background': '#F1FAEE',  # Light cream for the background
    'grid': '#A8DADC',  # Light teal for grid lines
    'title': '#1D3557',  # Dark blue for title
    'axes': '#457B9D'  # Steel blue for axes labels
}

# Create the interactive line plot with Plotly for body mass
fig = px.line(df_body_mass, x='startDate', y='value', 
              title='Body Mass Over Time', 
              labels={'value': 'Body Mass (lb)', 'startDate': 'Date'})

# Update layout to match the color scheme and center the title
fig.update_layout(
    plot_bgcolor=colors['background'],  # Set background color
    paper_bgcolor=colors['background'],  # Paper (overall background)
    title_font=dict(size=24, color=colors['title'], family="Arial"),  # Title font styling
    title_x=0.5,  # Center the title
    xaxis=dict(tickformat='%Y-%m-%d', gridcolor=colors['grid'], tickangle=-45),  # Combine x-axis settings
    yaxis=dict(gridcolor=colors['grid']),  # Set gridline color for y-axis
    font=dict(color=colors['axes']),  # Set font color for axes labels
    xaxis_title='Date',
    yaxis_title='Body Mass (lb)',
    showlegend=False  # Hide the legend
)

# Static line color
fig.update_traces(line_color=colors['line'], line_width=3)

# Show interactive plot
fig.show()


In [31]:
import pandas as pd
import plotly.express as px

# Load the workout data for running, swimming, and cycling
base_path = '/Users/ngirmay/Documents/GitHub/ironman_retrospective/apple_health/health_data_exported/'

# Load each workout file
df_swimming = pd.read_csv(base_path + 'HKWorkoutActivityTypeSwimming_2024-04-99_18-31-37_SimpleHealthExportCSV.csv', sep=',', skiprows=1)
df_running = pd.read_csv(base_path + 'HKWorkoutActivityTypeRunning_2024-04-99_18-31-37_SimpleHealthExportCSV.csv', sep=',', skiprows=1)
df_cycling = pd.read_csv(base_path + 'HKWorkoutActivityTypeCycling_2024-04-99_18-31-36_SimpleHealthExportCSV.csv', sep=',', skiprows=1)

# Convert 'startDate' to datetime
df_swimming['startDate'] = pd.to_datetime(df_swimming['startDate'])
df_running['startDate'] = pd.to_datetime(df_running['startDate'])
df_cycling['startDate'] = pd.to_datetime(df_cycling['startDate'])

# Plot duration for each workout type
fig_swimming = px.line(df_swimming, x='startDate', y='duration', title='Swimming Duration Over Time', labels={'duration': 'Duration (min)'})
fig_running = px.line(df_running, x='startDate', y='duration', title='Running Duration Over Time', labels={'duration': 'Duration (min)'})
fig_cycling = px.line(df_cycling, x='startDate', y='duration', title='Cycling Duration Over Time', labels={'duration': 'Duration (min)'})

# Show the plots
fig_swimming.show()
fig_running.show()
fig_cycling.show()
